In [1]:
!pwd

/Users/christos/FourthBrain/code/mle11-spotify1/RealTime-Custom-Keyword-Spotting/Code


In [2]:
# code to find unused files in a project
# import os

# # Define the project directory
# project_dir = "/Users/christos/FourthBrain/code/mle11-spotify1/RealTime-Custom-Keyword-Spotting"

# # Find all Python files in the project directory
# python_files = []
# for dirpath, dirnames, filenames in os.walk(project_dir):
#     for filename in filenames:
#         if filename.endswith(".py"):
#             python_files.append(os.path.join(dirpath, filename))

# # Find all used files and scripts in the project
# used_files = []
# for python_file in python_files:
#     with open(python_file, "r") as f:
#         lines = f.readlines()
#         for line in lines:
#             # if line.startswith("import") or line.startswith("from"):
#             #     split_line = line.split()
#             #     print(line)
#             if line.startswith("import"):
#                 split_line = line.split()
#                 print(line)
#                 if len(split_line) > 1:
#                     used_file = split_line[1]
#                     if used_file.endswith(".py"):
#                         used_files.append(used_file)

# Find all unused files and scripts in the project
# unused_files = []
# for python_file in python_files:
#     if python_file not in used_files:
#         unused_files.append(python_file)

# print("Unused files and scripts:")
# for unused_file in unused_files:
#     print(unused_file)


In [2]:
# import lines starting with import
import argparse
import asyncio
import base64
import collections
import datetime
import glob
import IPython
import importlib
import joblib
import json
import logging
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import pandas as pd
import pathlib
import pickle
import plotly.express as px
import plotly.graph_objects as go
import pyaudio
import seaborn as sns
import sox
import streamlit as st
import struct
import subprocess
import sys
import tensorflow as tf
import time
import websockets

# all the from imports
from dataclasses import dataclass
from dotenv import load_dotenv
from fastapi import FastAPI, Query, HTTPException
from pathlib import Path
from pyannote.audio import Pipeline # important from recording
from pydantic import BaseModel
from scipy.io import wavfile
from scipy.io.wavfile import write
from sklearn.model_selection import train_test_split
from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op
from tensorflow.python.platform import gfile
from tqdm.notebook import tqdm
from typing import Dict, List, Optional

# deep dive into func folder
from func import audio_record, kws_train, transfer_learning, input_data, spk_segment, inference, batch_streaming_analysis as sa
from func.single_target_recognize_commands import SingleTargetRecognizeCommands, RecognizeResult
from func.accuracy_utils import StreamingAccuracyStats
# test_streaming_accuracy.py file not needed

# model and model_realtime python files being used
from model_realtime import record, preproc, train, predict, report_result, infer
from model import train,predict, report_result


/Users/christos/opt/miniconda3/envs/kws2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# use cProfile to find out which functions are taking the most time and which are not being used
import cProfile
import pstats
import subprocess

def entire_app():
    # use this string to run the entire app
    # executable = "streamlit run streamlit_app.py"
    # execute string
    # subprocess.call(executable, shell=True)
    KEYWORD='heyBebe'
    keyword_dir = record(KEYWORD)
    spk_segments_path = preproc(KEYWORD,keyword_dir)
    print("Record completed! 🏁")
    test_samples= train(KEYWORD, spk_segments_path)
    print("Training completed 🏁")
    target_pred, nontarget_pred = predict(KEYWORD, test_samples)
    frr_val,far_val = report_result(target_pred, nontarget_pred)
    print("FRR and FAA are:", frr_val, far_val)
    result = infer(KEYWORD,duration=10)
    return result


cProfile.run('entire_app()', 'restats')

In [14]:
# read restats file
p = pstats.Stats('restats')
# sort by time
p.print_stats()

Thu Mar 23 13:40:28 2023    restats

         143 function calls in 200.957 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {method 'rfind' of 'bytes' objects}
        1    0.000    0.000    0.000    0.000 {method 'rstrip' of 'bytes' objects}
        1    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
        1    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'rstrip' of 'str' objects}
        1    0.000    0.000  200.957  200.957 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}


In [9]:
import cProfile

def my_function():
    for i in range(10000):
        pass

cProfile.run('my_function()')

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 1407773237.py:3(my_function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


